In [1]:
from tqdm import tqdm
import os
from datasets import load_dataset, concatenate_datasets
import json
import re
from copy import deepcopy

import asyncio
import math
import numpy as np
import random
import pandas as pd
from ollama import AsyncClient

from random import sample
from collections import Counter
from openai import OpenAI
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from utils import *

/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [138]:
system_prompt = f'''You are an expert in agronomy, skilled at categorizing titles based on their topics. I will provide you with a WikiHow title for an agronomy procedure and with a list of possible categories. Your task is to assign one of the categories provided to the procedure or come up with a new one if you don't find a match.
Carefully review the title to determine the most appropriate category it belongs to.  Your answer should start with a single line containing only the name of the chosen category.
'''

In [139]:
#SGLANG
import openai
client_sglang = openai.AsyncClient(
    base_url="http://192.168.1.5:30000/v1", api_key="EMPTY")

async def chat_sglang(content):
    
    response = await client_sglang.chat.completions.create(
    model="default",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": content},
    ],
    temperature=0.0,
    max_tokens=512,
    )
    return response.choices[0].message.content

### Load the Wikihow dataset

In [53]:
wifi_dataset=load_json('/workdir/wikihow/wikihow.json')

In [54]:
len(wifi_dataset)

2141

In [45]:
wifi_dataset[0]['question'].split('"')[0]+wifi_dataset[0]['question'].split('"')[-2]+wifi_dataset[0]['question'].split('"')[-1]

'Write down all the steps from the Prune an Olive Tree procedure.'

In [58]:
title = set()
for doc in wifi_dataset:
    section_title = doc['section_title']
    page_title = doc['page_title']
    title.add(f"{page_title}")#: {section_title}")
    if section_title=='Steps':
        new_question=doc['question'].split('"')[0]+doc['question'].split('"')[-2]+doc['question'].split('"')[-1]
        print(doc['question'])
        print()
        print(new_question)
        print()

Put these steps in the correct order for the "Steps" instructions in the "Turn Clay Into Growing Soil" guide.

Steps: 1. Consider using a wintercover cropsuch as annual rye or anylegume.
2. Get your soil tested.
3. Add the suggested amendments to your soil.
4. Clay soil may also need some secondary macronutrients, such as calcium (Ca), sulfur (S), and  magnesium (Mg).
5. When starting new beds, consider usingthe "lasagna" methodto prepare the soil..

Put these steps in the correct order for the lasagna methodto prepare the soil..

Put these steps in the correct order for the "Steps" instructions in the "Dye White Roses with Food Coloring" guide.

Steps: 1. Remove all leaves from the roses.
2. Measure approximately 30-60 mL (1-2 fl.
3. Add 2-4 mL (⅓ to ⅔ tsp) of food coloring and let it disperse throughout the water until the color is nearly uniform.
4. Place the rose(s) in the colored water for up to a week.
5. Gather the white roses, food coloring, and container.
6. Cut the rose stem(

### categorization

In [131]:
def get_prompt(title, prompt, categories=[]):
    return '[Title]\n'+title+'\n'+'[End of Title]\n'+'[Categories]\n'+str(categories)+'\n[End of Categories]\n'+prompt

In [132]:
prompt = '''Try to assign one category from the list of categories to the previous procedure title, if the title is not related to any of them create a new category. 
This procedure title is taken from WikiHow.
Your answer should start with one line containing only the category name.'''

In [133]:
print(get_prompt(list(title)[0],prompt))

[Title]
Harvest Pineapple
[End of Title]
[Categories]
[]
[End of Categories]
Try to assign one category from the list of categories to the previous procedure title, if the title is not related to any of them create a new category. 
This procedure title is taken from WikiHow.
Your answer should start with one line containing only the category name.


In [136]:
list(title)[0]

'Harvest Pineapple'

In [137]:
await chat_sglang(get_prompt(list(title)[0],prompt))

'Cooking'

In [118]:
d = defaultdict(list)

In [121]:
list(d.keys())

[]

In [145]:
from collections import defaultdict
#categories_dict = defaultdict(list)
categories_dict = {k:[] for k in categories_dict}
extra_dict = {'suggestions' : [], 'catastrophies' : []}

#end_prompt =  'Assign a category to the question above. Your answer should include only the with a single line containing only the name of the chosen category.'


for t in tqdm(list(title)):
    categories=list(categories_dict.keys())
    response = await chat_sglang(get_prompt(t, prompt, categories))
    try:
        category = response.split('\n')[0].strip()
        if category not in categories:
            print('Suggestion: ', response)
            extra_dict['suggestions'].append((response,t))
            category = response.split('\n')[-1].strip()
            if category not in categories:
                print('Category not present')
                print(f"Category: {category} | {t}")
        #categories.add(category)
        categories_dict[category].append(t)
    except:
        extra_dict['catastrophies'].append((response,t))
        print('Catastrophy', response)

100%|██████████| 935/935 [01:25<00:00, 10.88it/s]


In [146]:
for k,v in categories_dict.items():
    print(k,len(v))

Fruit Production 78
Ornamental Tree Care 172
Houseplant Care 155
Grain Production 21
Nut Tree Care 13
Soil Improvement 49
Vegetable Production 114
Wildlife Management 34
Herb Production 89
Weed Control 43
Landscape Maintenance 129
Pest Control 38


In [151]:
save_json(categories_dict,'wikihow_LLM_category.json')

In [148]:
categories_dict

{'Fruit Production': ['Harvest Pineapple',
  'Grow Banana Plants',
  'Grow Passion Fruit',
  'Harvest Olives',
  'Prune Apricot Trees',
  'Prune Gooseberry',
  'Germinate Pomegranate Seeds',
  'Grow a Pomegranate Tree',
  'Grow a Square Watermelon',
  'Grow a Pineapple',
  'Prune a Fig Tree',
  'Prune Citrus Trees',
  'Plant Guava Trees',
  'Prune Plum Trees',
  'Grow Cranberries',
  'Prune Apple Trees',
  'Grow a Choko Vine',
  'Graft a Fruit Tree',
  'Prune a Satsuma Tree',
  'Prepare the Soil for Strawberries',
  'Plant Apple Seeds',
  'Pollinate a Lemon Tree',
  'Prune Banana Trees',
  'Prune Guava Trees',
  'Prune Old Apple Trees',
  'Stake an Apple Tree',
  'Prune Grape Vines',
  'Grow a Coconut Tree',
  'Grow Grape Vines',
  'Grow Kumquat',
  'Grow Strawberries from the Seed',
  'Grow Grape Vine Cuttings',
  'Plant a Plum Seed',
  'Grow Avocados',
  'Pick Grapefruit',
  'Plant Dragon Fruit',
  'Grow an Apple Tree from a Seed',
  'Plant Loquat Seeds',
  'Harvest Blackcurrants',
 

In [129]:
set(results)

{'Animal Husbandry',
 'Aquaculture',
 'Aquarium Care',
 'Aquatic Plant Care',
 'Beekeeping',
 'Business Management',
 'Composting',
 'Composting and Fertilizers',
 'Conservation and Preservation',
 'Conservation and Wildlife Management',
 'Container Gardening',
 'Crafts',
 'Crop Cultivation',
 'Crop Diseases',
 'Crop Identification',
 'Crop Management',
 'Crop Protection',
 'Cultivating and Harvesting',
 'Decorative Arts',
 'Disease Management',
 'Diseases and Pests',
 'Diseases and Pests Management',
 'Farming and Agriculture',
 'Fertilization',
 'Fertilizers',
 'Floral Arrangement',
 'Floral Care',
 'Floriculture',
 'Floristry',
 'Flower Arrangement',
 'Flower Care',
 'Flower Gardening',
 'Food Preservation',
 'Foraging',
 'Forcing Tulip Bulbs in Water\n\nSince there are no categories provided, I have created a new one based on the procedure title.',
 'Forestry',
 'Fruit Cultivation',
 'Fruit Gardening',
 'Fruit Growing',
 'Fruit Harvesting',
 'Fruit Selection',
 'Fruit Tree Care',
 

In [3]:
categories=load_json('/workdir/wikihow/wikihow_category.json')

In [5]:
list(categories.keys())

['Gardening',
 'Gardening/Cleaning-Up-Garden',
 'Gardening/Compost-Mulch-and-Soil-Preparation',
 'Gardening/Compost-Mulch-and-Soil-Preparation/Compost-Containers',
 'Gardening/Compost-Mulch-and-Soil-Preparation/Fertilizers',
 'Gardening/Compost-Mulch-and-Soil-Preparation/Soil-Chemistry',
 'Gardening/Creating-Habitats-for-Wildlife',
 'Gardening/Creating-Habitats-for-Wildlife/Attracting-Birds',
 'Gardening/Garden-Crafts',
 'Gardening/Garden-Crafts/Lavender-Crafts',
 'Gardening/Garden-Crafts/Plant-Gifts',
 'Gardening/Garden-Crafts/Rose-Crafts',
 'Gardening/Garden-Pests-and-Weeds',
 'Gardening/Garden-Pests-and-Weeds/Garden-Sprays',
 'Gardening/Garden-Projects',
 'Gardening/Garden-Projects/Greenhouses',
 'Gardening/Garden-Projects/Scarecrows',
 'Gardening/Gardening-Tools',
 'Gardening/Gardening-Tools/Lawn-Mowers',
 'Gardening/Harvest-and-Storage',
 'Gardening/Hydroponics',
 'Gardening/Indoor-and-Patio-Plants',
 'Gardening/Landscaping-Plants',
 'Gardening/Landscaping-Plants/Growing-Grasses',